# **Connect to Github**
- This method works given the FanTrack codebase are stored in Github repository
- If reset session, you need to update the key in github: https://github.com/settings/keys
- https://stackoverflow.com/questions/13363553/git-error-host-key-verification-failed-when-connecting-to-remote-repository

In [ ]:
!ssh-keygen

In [ ]:
!cat /root/.ssh/id_rsa.pub

In [ ]:
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# **Environment Setup**
- Set up environment suitable for running FanTrack in TensorFlow 1.13.1
- Set up dependent files for AVOD


In [ ]:
# This is to connect to where KITTI dataset resids
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/

In [ ]:
!git clone -b copy git@github.com:sarauc/fantrack.git --recurse-submodules
#!git clone https://github.com/sarauc/fantrack --recurse-submodules

In [ ]:
%cd /content/fantrack
!tail .gitmodules

In [ ]:
!git submodule update --init --recursive

In [ ]:
#deal with error that cannot access xx folder
#https://medium.com/@cwcgchello/how-to-set-pythonpath-in-colab-bf8c239c7c29
import os
os.environ['PYTHONPATH'] += ":/content/fantrack/"
os.environ['PYTHONPATH'] += ":/content/fantrack/object_detector"
os.environ['PYTHONPATH'] += ":/content/fantrack/object_detector/wavedata"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/object_detector/avod/"

!echo $PYTHONPATH

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!conda install cudatoolkit=10.0 python=3.6.5 -y
!conda install -c anaconda cudnn=7.6.4 -y

In [ ]:
#only run once
!pip install -r /content/fantrack/requirements.txt
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1
!pip install protobuf==3.20.0
!pip install ipykernel

In [ ]:
%%python3
# After previous block finishes running, restart the session and run again
# Make sure TF version is 1.13
import tensorflow as tf 
print(tf.__version__)

In [ ]:
%cd /content/fantrack/fan_track/object_detector/ 

In [ ]:
!git clone git@github.com:sarauc/avod.git --recurse-submodules

In [ ]:
# move /object_detector/avod/* to /object_detector/
!mv /content/fantrack/fan_track/object_detector/avod/ /content/fantrack/fan_track/object_detector/tmp_avod/
!cp -r /content/fantrack/fan_track/object_detector/tmp_avod/* /content/fantrack/fan_track/object_detector/
!rm -r /content/fantrack/fan_track/object_detector/tmp_avod/

In [ ]:
!git submodule update --init --recursive

In [ ]:
#deal with error that cannot access xx folder
#https://medium.com/@cwcgchello/how-to-set-pythonpath-in-colab-bf8c239c7c29
import os
os.environ['PYTHONPATH'] += ":/content/fantrack/"
os.environ['PYTHONPATH'] += ":/content/fantrack/object_detector"
os.environ['PYTHONPATH'] += ":/content/fantrack/object_detector/wavedata"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/object_detector/avod/"

!echo $PYTHONPATH

In [ ]:
!sh scripts/install/build_integral_image_lib.bash

In [ ]:
!sh avod/protos/run_protoc.sh

In [ ]:
# Need to change the fantrack script 
'''line 654
<     def create_feed_dict(self):
<         feed_dict = self._rpn_model.create_feed_dict()
---
    def create_feed_dict(self, sample_index=None):
        feed_dict = self._rpn_model.create_feed_dict(sample_index)
        self.sample_info = self._rpn_model.sample_info
        return feed_dict
'''


# /content/fantrack/fan_track/object_detector/avod/core/models/avod_model.py

# **Train AVOD**

In [ ]:
#https://medium.com/@cwcgchello/how-to-set-pythonpath-in-colab-bf8c239c7c29
import os
os.environ['PYTHONPATH'] += ":/content/fantrack/"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/object_detector"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/object_detector/wavedata/wavedata"
os.environ['PYTHONPATH'] += ":/content/fantrack/fan_track/object_detector/wavedata/"

!echo $PYTHONPATH

In [ ]:
# add pretrained weights and configs
!mkdir /content/fantrack/fan_track/object_detector/data/
!mkdir /content/fantrack/fan_track/object_detector/data/outputs/
!cp /content/drive/MyDrive/KITTI/avod_weights/avod_cars_fast.zip /content/fantrack/fan_track/object_detector/data/

In [ ]:
!unzip /content/fantrack/fan_track/object_detector/data/avod_cars_fast.zip -d /content/fantrack/fan_track/object_detector/data/

In [ ]:
# copy pretrained model to local
!cp /content/drive/MyDrive/Kitti/vgg_16.ckpt /content/fantrack/fan_track/avod_feature_extractor/checkpoints/img_vgg/
#!cp /content/Kitti/vgg_16.ckpt /content/fantrack/fan_track/avod_feature_extractor/checkpoints/img_vgg/

In [ ]:
# copy checkpoint file to local
!cp -r /content/drive/MyDrive/Fantrack_7videos_withBN/fantrack_v3/data /content/fantrack/fan_track

# **Train SimNet**

In [ ]:
# copy data (50GB ~ 50m)
!cp -r /content/drive/MyDrive/Kitti/ /content/Kitti/

Below contains a list of issues with original code, and their corresponding modifications. If one issue is indicated "updated in script", then no need to edit again in the current session. But if not, modifications in codes are needed. 

In [ ]:
# Before run 
# 1. (updated in script) NameError: name 'AssocnetConfig' is not defined 
# /content/fantrack/fan_track/experiments/train_simnet.py
#  line 22-23
'''
    if not os.path.exists(SimnetConfig.SIMNET_CKPT_DIR):
        os.makedirs(SimnetConfig.SIMNET_CKPT_DIR)
'''

# 2. (updated in script) FileNotFoundError: [Errno 2] No such file or directory: '/content/fantrack/fan_track/object_detector/avod/data/outputs/pyramid_cars_with_aug_example/pyramid_cars_with_aug_example.config'
# modify /content/fantrack/fan_track/data_generation/kitti_simnet_dataset.py
# line 43- line 47 
'''
avod_root_dir = '/content/fantrack/fan_track/object_detector/'
checkpoint_name = 'avod_cars_fast'
experiment_config = checkpoint_name + '.config'
experiment_config_path = os.path.join(avod_root_dir ,'data',checkpoint_name,experiment_config)
experiment_config_path
'''

# 3. (updated in script) FileNotFoundError: Dataset path does not exist: /root/Kitti/object
'''
modify /content/fantrack/fan_track/config/config.py line 14: KITTI_ROOT = '/content/Kitti'
'''

# 4. float issue (must update even if simnet is not run:assocnet will also need it)
'''
# - create 4 empty folders and copy planes from training folder
# - insert p = p[:12] in line 92 before 'float 'in /content/fantrack/fan_track/object_detector/wavedata/wavedata/tools/core/calib_utils.py
# - insert tr_rect = tr_rect[:9] in line 106 in same file
# - insert tr_v2c = tr_v2c[:12] in line 112 in same file
'''

# 5. (updated in script) velodyne: 002-177-179 are missing: https://github.com/aurora-opensource/xviz/issues/391
'''
set line 90 to true for debug:
# /content/fantrack/fan_track/data_generation/simnet_batch_sampler.py
'''

# 6. only edit for quickly debug purpose:
# copy below code to line 250: /content/fantrack/fan_track/data_generation/kitti_simnet_dataset.py
#  --> create_examples_odd_even
'''
                                        print(video_no,example_idx,frame_id ) 
                                        # debug save only first 3 frames
                                        if frame_id >= 3:
                                           print (total_pos_exp_video, total_neg_exp_video)
                                           return total_pos_exp_video, total_neg_exp_video
                                        if frame_id >= 2:
                                           is_training_frame = False
'''

# copy below code to line 420 (before example_idx, num_pos_exp,num_neg_pos,_ = self.save_examples(video_no,)
# --> create_examples

'''
                            # debug save only first 5 frames
                            if frame_id >= 3:
                                print (total_pos_exp_video, total_neg_exp_video)
                                return total_pos_exp_video, total_neg_exp_video
                            if frame_id >= 2:
                               is_training_frame = False
'''

# 7. only edit for purposes of training part-of-video 
# Run full sequence of video 0-9 
# /content/fantrack/fan_track/data_generation/kitti_simnet_dataset.py
''' line 99
            if int(video_no) == 10:
              break
'''
# /content/fantrack/fan_track/data_generation/kitti_assocnet_dataset.py
''' line 102
            if int(video_no) == 10:
              break
'''

# 8. Only edit if BN layer needs removing
# Remove BN layer.
''' /content/fantrack/fan_track/network/model.py
line 339
            with slim.arg_scope([slim.fully_connected, slim.conv2d],
                                weights_regularizer = w_regularizer,
                                activation_fn = tf.nn.leaky_relu,
                                normalizer_fn = None #batch_norm_fn,
                               ) as arg_scope:
line 541
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                            activation_fn = tf.nn.leaky_relu,
                            weights_regularizer = w_regularizer,
                            weights_initializer = w_initializer,
                            normalizer_fn = None #slim.batch_norm,
                           ) as arg_scope:
'''

# 9. Only edit for purpose of training shorter epochs
# reduce epoch number
''' 
/content/fantrack/fan_track/experiments/train_simnet.py
line 20
    args.epochs = 100
/content/fantrack/fan_track/config/config.py
line 45
    EPOCHS = 100

'''

In [ ]:
# create empty folder in object/testing/ 
# copy planes from training set
!cp -r /content/Kitti/object/training/planes /content/Kitti/object/testing/
!mkdir /content/Kitti/object/testing/calib/
!mkdir /content/Kitti/object/testing/image_2/
!mkdir /content/Kitti/object/testing/label_2/
!mkdir /content/Kitti/object/testing/velodyne/

In [ ]:
%cd /content/fantrack/
!python3 fan_track/experiments/train_simnet.py

# **Train AssoNet**

In [ ]:
# 1. (updated in script) FileNotFoundError: [Errno 2] No such file or directory: '/content/fantrack/fan_track/object_detector/avod/data/outputs/pyramid_cars_with_aug_example/pyramid_cars_with_aug_example.config'
''' 
/content/fantrack/fan_track/data_generation/kitti_assocnet_dataset.py
line55-59 before read the configuration

        avod_root_dir = '/content/fantrack/fan_track/object_detector/'
        checkpoint_name = 'avod_cars_fast'
        experiment_config = checkpoint_name + '.config'
        experiment_config_path = os.path.join(avod_root_dir ,'data',checkpoint_name,experiment_config)
        experiment_config_path
'''

# 2. Only edit for quickly debug purposes
'''
/content/fantrack/fan_track/data_generation/kitti_assocnet_dataset.py line 180
                                            # debug save only first 3 frames
                                            if frame_id > 3:
                                               train_val = 'validation'
                                            if frame_id >= 5:
                                               continue
'''

# 3. Change config of Simnet if needed

'''/content/fantrack/fan_track/config/config.py
line 44 
    SIMNET_CKPT_PATH = os.path.join(SIMNET_CKPT_DIR, 'simnet.ckpt-99')

'''


In [ ]:
%cd /content/fantrack/
!python3 fan_track/experiments/train_assocnet.py

# **Run Inference**

In [ ]:
# 1. (updated in script) Change AVOD directory
'''
 /content/fantrack/fan_track/network/tracker.py
line361-366 after initialize AVOD

		avod_root_dir = '/content/fantrack/fan_track/object_detector/'
	  
		avod_checkpoint = os.path.join(avod_root_dir, 'data', \
						  args.avod_checkpoint_name,'checkpoints', args.avod_checkpoint_name + '-' + TrackerConfig.AVOD_CKPT_NUMBER)
		experiment_config_path = os.path.join(avod_root_dir ,'data', \
              args.avod_checkpoint_name, args.avod_checkpoint_name + '.config')

/content/fantrack/fan_track/config/config.py
Edit checkpoint number for ASSOCNET and Tracker
line 44
    SIMNET_CKPT_PATH = os.path.join(SIMNET_CKPT_DIR, 'simnet.ckpt-99')

line 62
    ASSOCNET_CKPT_PATH = os.path.join(ASSOCNET_CKPT_DIR, 'assocnet.ckpt-99')

line 91
    AVOD_CKPT_NAME = 'avod_cars_fast'
    AVOD_CKPT_NUMBER = '00093000'

line 19: /content/fantrack/fan_track/experiments/run_tracker.py
    args.avod_checkpoint_name = 'avod_cars_fast'

'''
# 2. edit for purposes of inferencing only on two videos
'''/content/fantrack/fan_track/experiments/run_tracker.py
line 27
    for i in range(2):

'''

# 3. save inference data and results
'''/content/fantrack/fan_track/network/tracker.py
line 530
				associations = self.get_associations(targets, measurements, self.video_no, self.current_frame_name)

line 894
	def get_associations(self, targets, measurements, video_no, frame_id):

line 1066
		# save to local
		example = {
        'target_centers': target_centers,
        'num_targets': targets,
        'm_centers': self.mapper.m_centers,
        'm_pred_x':m_pred_x,
        'm_pred_y': m_pred_y,
        'local_corr_map':local_corr_map,
        'associations':associations,
        'corr_scores': corr_scores,
    }

		save_path = '/content/demo/data/%s/'%video_no
    # create folder 
		# Render results
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		filename = os.path.join(save_path, 'infer_data_video_%s_frame_%s.npy'%(video_no, frame_id))
		np.save(filename, example)
'''


In [ ]:
%cd /content/fantrack/
!python3 fan_track/experiments/run_tracker.py